In [1]:
# Set local python and nltk paths
import sys
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages')
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg')
sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/IPython/extensions')
sys.path

['/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/IPython/extensions',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages',
 '',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python27.zip',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/plat-linux2',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/lib-tk',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/lib-old',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/lib-dynload',
 '/homes/ag6516/.local/lib/python2.7/site-packages',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages',
 '/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python

In [2]:
%matplotlib inline

import os
import pickle

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import skimage.transform
import random
from random import randint
random.seed(123)
np.random.seed(123)

# Set THEANO_FLAGS='device=cuda0,floatX=float32' to run notebook on gpu
import lasagne
from lasagne.utils import floatX
import theano
import theano.tensor as T
from collections import Counter
from lasagne.utils import floatX

import googlenet

Using cuDNN version 5110 on context None
Mapped name None to device cuda0: GeForce GTX 1050 (0000:01:00.0)


In [3]:
dir = ('/vol/medic02/users/ag6516/x_ray_fracture_localisation/')
# dir = ('/Users/Aydan/PhD/x_ray_fracture_localisation/')

In [4]:
sample_images = pickle.load(open('sample_exams_train_test_split.pkl'))
print len(sample_images)

1432


In [5]:
allwords = Counter()
for item in sample_images:
    tokens = item['tokens']
    #for sentence in item['sentences']:
    allwords.update(tokens)
        
vocab = [k for k, v in allwords.items()]
vocab.insert(0, '#START#')
vocab.append('#END#')

word_to_index = {w: i for i, w in enumerate(vocab)}
index_to_word = {i: w for i, w in enumerate(vocab)}

len(vocab)

107

## Extract CNN Features from GoogLeNet

In [6]:
cnn_layers = googlenet.build_model()
cnn_input_var = cnn_layers['input'].input_var
cnn_feature_layer = cnn_layers['pool5/7x7_s1']
cnn_output_layer = cnn_layers['prob']

get_cnn_features = theano.function([cnn_input_var], lasagne.layers.get_output(cnn_feature_layer))

In [7]:
model_param_values = pickle.load(open('models/blvc_googlenet.pkl'))['param values']
lasagne.layers.set_all_param_values(cnn_output_layer, model_param_values)

In [8]:
MEAN_VALUES = np.array([104, 117, 123]).reshape((3,1,1))

def prep_image(im):
    if len(im.shape) == 2:
        im = im[:, :, np.newaxis]
        im = np.repeat(im, 3, axis=2)
    # Resize so smallest dim = 224, preserving aspect ratio
    h, w, _ = im.shape
    if h < w:
        im = skimage.transform.resize(im, (224, w*224/h), preserve_range=True)
    else:
        im = skimage.transform.resize(im, (h*224/w, 224), preserve_range=True)

    # Central crop to 224x224
    h, w, _ = im.shape
    im = im[h//2-112:h//2+112, w//2-112:w//2+112]
    
    rawim = np.copy(im).astype('uint8')
    
    # Shuffle axes to c01
    im = np.swapaxes(np.swapaxes(im, 1, 2), 0, 1)
    
    # Convert to BGR
    im = im[::-1, :, :]

    im = im - MEAN_VALUES
    return rawim, floatX(im[np.newaxis])

In [9]:
for item in sample_images:
    path = dir + 'data/Images/' + item['impath']
    #try:
    im = plt.imread(path)
    _, cnn_input = prep_image(im)
    #except IOError:
    #        continue
    features = get_cnn_features(cnn_input)
    item['cnn features'] = features

/vol/medic02/users/ag6516/miniconda/envs/TheanoLasagne/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [10]:
pickle.dump(sample_images, open('sample_images_baseline.pkl','w'), protocol=pickle.HIGHEST_PROTOCOL)

## Report Generation Model - Single Image Input

In [11]:
sample_images = pickle.load(open('sample_images_baseline.pkl'))
print len(sample_images)

1432


In [12]:
SEQUENCE_LENGTH = 33
MAX_SENTENCE_LENGTH = SEQUENCE_LENGTH - 3 # 1 for image, 1 for start token, 1 for end token
BATCH_SIZE = 10
CNN_FEATURE_SIZE = 1024
EMBEDDING_SIZE = 256

In [13]:
# Returns a list of tuples (cnn features, list of words, image ID)
def get_data_batch(dataset, size, split='train'):
    items = []
    
    while len(items) < size:
        item = random.choice(dataset)
        if item['split'] != split:
            continue
        sentence = item['tokens']
        if len(sentence) > MAX_SENTENCE_LENGTH:
            sentence = sentence[1:MAX_SENTENCE_LENGTH]
            #continue
        items.append((item['cnn features'], sentence, item['imid']))
    
    return items

In [14]:
# Convert a list of tuples into arrays that can be fed into the network
def prep_batch_for_network(batch):
    x_cnn = floatX(np.zeros((len(batch), CNN_FEATURE_SIZE)))
    x_sentence = np.zeros((len(batch), SEQUENCE_LENGTH - 1), dtype='int32')
    y_sentence = np.zeros((len(batch), SEQUENCE_LENGTH), dtype='int32')
    mask = np.zeros((len(batch), SEQUENCE_LENGTH), dtype='bool')

    for j, (cnn_features, sentence, _) in enumerate(batch):
        x_cnn[j] = cnn_features
        i = 0
        for word in ['#START#'] + sentence + ['#END#']:
            if word in word_to_index:
                mask[j, i] = True
                y_sentence[j, i] = word_to_index[word]
                x_sentence[j, i] = word_to_index[word]
                i += 1
        #mask[j, 0] = False
                
    return x_cnn, x_sentence, y_sentence, mask

In [15]:
# sentence embedding maps integer sequence with dim (BATCH_SIZE, SEQUENCE_LENGTH - 1) to 
# (BATCH_SIZE, SEQUENCE_LENGTH-1, EMBEDDING_SIZE)
l_input_sentence = lasagne.layers.InputLayer((BATCH_SIZE, SEQUENCE_LENGTH - 1))
l_sentence_embedding = lasagne.layers.EmbeddingLayer(l_input_sentence,
                                                     input_size=len(vocab),
                                                     output_size=EMBEDDING_SIZE,
                                                    )

# cnn embedding changes the dimensionality of the representation from 1024 to EMBEDDING_SIZE, 
# and reshapes to add the time dimension - final dim (BATCH_SIZE, 1, EMBEDDING_SIZE)
l_input_cnn = lasagne.layers.InputLayer((BATCH_SIZE, CNN_FEATURE_SIZE))
l_cnn_embedding = lasagne.layers.DenseLayer(l_input_cnn, num_units=EMBEDDING_SIZE,
                                            nonlinearity=lasagne.nonlinearities.identity)

l_cnn_embedding = lasagne.layers.ReshapeLayer(l_cnn_embedding, ([0], 1, [1]))

# the two are concatenated to form the RNN input with dim (BATCH_SIZE, SEQUENCE_LENGTH, EMBEDDING_SIZE)
l_rnn_input = lasagne.layers.ConcatLayer([l_cnn_embedding, l_sentence_embedding])


l_dropout_input = lasagne.layers.DropoutLayer(l_rnn_input, p=0.5)
l_lstm = lasagne.layers.LSTMLayer(l_dropout_input,
                                  num_units=EMBEDDING_SIZE,
                                  unroll_scan=True,
                                  grad_clipping=5.)
l_dropout_output = lasagne.layers.DropoutLayer(l_lstm, p=0.5)

# the RNN output is reshaped to combine the batch and time dimensions
# dim (BATCH_SIZE * SEQUENCE_LENGTH, EMBEDDING_SIZE)
l_shp = lasagne.layers.ReshapeLayer(l_dropout_output, (-1, EMBEDDING_SIZE))

# decoder is a fully connected layer with one output unit for each word in the vocabulary
l_decoder = lasagne.layers.DenseLayer(l_shp, num_units=len(vocab), nonlinearity=lasagne.nonlinearities.softmax)

# finally, the separation between batch and time dimension is restored
l_out = lasagne.layers.ReshapeLayer(l_decoder, (BATCH_SIZE, SEQUENCE_LENGTH, len(vocab)))

In [16]:
# cnn feature vector
x_cnn_sym = T.matrix()

# sentence encoded as sequence of integer word tokens
x_sentence_sym = T.imatrix()

# mask defines which elements of the sequence should be predicted
mask_sym = T.imatrix()

# ground truth for the RNN output
y_sentence_sym = T.imatrix()

In [17]:
output = lasagne.layers.get_output(l_out, {
                l_input_sentence: x_sentence_sym,
                l_input_cnn: x_cnn_sym
                })

In [18]:
def calc_cross_ent(net_output, mask, targets):
    # Helper function to calculate the cross entropy error
    preds = T.reshape(net_output, (-1, len(vocab)))
    targets = T.flatten(targets)
    cost = T.nnet.categorical_crossentropy(preds, targets)[T.flatten(mask).nonzero()]
    return cost

loss = T.mean(calc_cross_ent(output, mask_sym, y_sentence_sym))

In [19]:
MAX_GRAD_NORM = 15

all_params = lasagne.layers.get_all_params(l_out, trainable=True)

all_grads = T.grad(loss, all_params)
all_grads = [T.clip(g, -5, 5) for g in all_grads]
all_grads, norm = lasagne.updates.total_norm_constraint(
    all_grads, MAX_GRAD_NORM, return_norm=True)

updates = lasagne.updates.adam(all_grads, all_params, learning_rate=0.001)

f_train = theano.function([x_cnn_sym, x_sentence_sym, mask_sym, y_sentence_sym],
                          [loss, norm],
                          updates=updates
                         )

f_val = theano.function([x_cnn_sym, x_sentence_sym, mask_sym, y_sentence_sym], loss)

In [20]:
for iteration in range(100000):
    x_cnn, x_sentence, y_sentence, mask = prep_batch_for_network(get_data_batch(sample_images, BATCH_SIZE))
    loss_train, norm = f_train(x_cnn, x_sentence, mask, y_sentence)
    if not iteration % 1000:
        print('Iteration {}, loss_train: {}, norm: {}'.format(iteration, loss_train, norm))
        try:
            batch = get_data_batch(sample_images, BATCH_SIZE, split='val')
            x_cnn, x_sentence, y_sentence, mask = prep_batch_for_network(batch)
            loss_val = f_val(x_cnn, x_sentence, mask, y_sentence)
            print('Val loss: {}'.format(loss_val))
        except IndexError:
            continue

Iteration 0, loss_train: 4.68226194382, norm: 0.594060063362
Val loss: 4.64461946487
Iteration 1000, loss_train: 1.49503552914, norm: 1.07000732422
Val loss: 1.44321966171
Iteration 2000, loss_train: 0.673087894917, norm: 1.06585860252
Val loss: 1.14956712723
Iteration 3000, loss_train: 0.540391325951, norm: 0.97417318821
Val loss: 0.658056914806
Iteration 4000, loss_train: 0.507909595966, norm: 0.824677884579
Val loss: 0.600943505764
Iteration 5000, loss_train: 0.527909398079, norm: 0.854552030563
Val loss: 1.52849411964
Iteration 6000, loss_train: 0.374013125896, norm: 0.650315344334
Val loss: 0.858984291553
Iteration 7000, loss_train: 0.39216119051, norm: 0.47963526845
Val loss: 0.435785651207
Iteration 8000, loss_train: 0.388289839029, norm: 0.533978760242
Val loss: 0.512874007225
Iteration 9000, loss_train: 0.383747458458, norm: 0.469817280769
Val loss: 0.533946633339
Iteration 10000, loss_train: 0.379906356335, norm: 0.70736438036
Val loss: 0.418379724026
Iteration 11000, loss_tr

Iteration 92000, loss_train: 0.320189803839, norm: 2.2265150547
Val loss: 0.956682860851
Iteration 93000, loss_train: 0.330505758524, norm: 2.19782519341
Val loss: 0.547796428204
Iteration 94000, loss_train: 0.306985139847, norm: 0.634790956974
Val loss: 0.511658072472
Iteration 95000, loss_train: 0.370091795921, norm: 1.72117245197
Val loss: 0.385338544846
Iteration 96000, loss_train: 0.279746085405, norm: 1.29769003391
Val loss: 0.391466408968
Iteration 97000, loss_train: 0.290550589561, norm: 1.76950073242
Val loss: 1.27965414524
Iteration 98000, loss_train: 0.346839666367, norm: 2.18422412872
Val loss: 0.444494456053
Iteration 99000, loss_train: 0.321956723928, norm: 3.51842212677
Val loss: 1.01565861702


In [21]:
param_values = lasagne.layers.get_all_param_values(l_out)
d = {'param values': param_values,
     'vocab': vocab,
     'word_to_index': word_to_index,
     'index_to_word': index_to_word,
    }
pickle.dump(d, open('baseline_trained_v_107.pkl','w'), protocol=pickle.HIGHEST_PROTOCOL)